In [41]:
import pyspark
from pyspark import SQLContext
import pyspark.sql.functions as F
import pyspark.sql.types as Types
sc = SparkContext.getOrCreate();
sql = SQLContext(sc)

citiesPopDf = (sql.read
         .format("com.databricks.spark.csv")
         .option("header", "true")
         .load("world_population/worldcities.csv"))\
         .withColumnRenamed("Population", "city_population")\
         .where(F.col("city_population").cast("int") > 0)

schema = Types.StructType([
    Types.StructField('Country/Region', Types.StringType(), True),
    Types.StructField('Population', Types.StringType(), True),
    Types.StructField('Urban Pop %', Types.StringType(), True),
    Types.StructField('World Share %', Types.StringType(), True),
    Types.StructField('Med. Age', Types.StringType(), True)
])

countriesPopDf = spark\
    .read\
    .json("world_population/population.json", schema, multiLine=True)\
    .withColumnRenamed("Population", "country_population")

joinedDf = citiesPopDf.join(countriesPopDf, (citiesPopDf.country == countriesPopDf["Country/Region"]))

joinedDf\
    .groupBy("country")\
    .count().sort(F.col("count").desc())\
    .limit(5)\
    .show()
"""
+-------------+-----+
|      country|count|
+-------------+-----+
|United States| 7328|
|       Russia|  564|
|        China|  392|
|       Brazil|  387|
|       Canada|  249|
+-------------+-----+
"""

joinedDf\
    .sort(F.col("country_population").cast("int").asc())\
    .groupBy("country")\
    .agg(\
         F.count("city").alias("Cities"),\
         F.sum(F.col("city_population").cast("int")).alias("urnab_pop"),\
         F.max(F.col("city_population").cast("int")).alias("most_pop_in_one_city"),\
         F.min(F.col("city_population").cast("int")).alias("min_pop_in_one_city"),\
         F.avg(F.col("city_population").cast("int")).alias("avg_city_pop")\
        )\
    .sort(F.col("urnab_pop").desc())\
    .limit(5)\
    .show()
"""
+-------------+------+---------+--------------------+-------------------+------------------+
|      country|Cities|urnab_pop|most_pop_in_one_city|min_pop_in_one_city|      avg_city_pop|
+-------------+------+---------+--------------------+-------------------+------------------+
|United States|  7328|390924051|            19354922|               1991|53346.622680131004|
|        China|   392|358546021|            14987000|                100| 914658.2168367347|
|        India|   212|204338075|            18978000|              10688| 963858.8443396227|
|       Brazil|   387|127259225|            18845000|                956| 328835.2067183463|
|        Japan|    69| 89712598|            35676000|              82335| 1300182.579710145|
+-------------+------+---------+--------------------+-------------------+------------------+
"""

+-------------+-----+
|      country|count|
+-------------+-----+
|United States| 7328|
|       Russia|  564|
|        China|  392|
|       Brazil|  387|
|       Canada|  249|
+-------------+-----+

+-------------+------+---------+--------------------+-------------------+------------------+
|      country|Cities|urnab_pop|most_pop_in_one_city|min_pop_in_one_city|      avg_city_pop|
+-------------+------+---------+--------------------+-------------------+------------------+
|United States|  7328|390924051|            19354922|               1991|53346.622680131004|
|        China|   392|358546021|            14987000|                100| 914658.2168367347|
|        India|   212|204338075|            18978000|              10688| 963858.8443396227|
|       Brazil|   387|127259225|            18845000|                956| 328835.2067183463|
|        Japan|    69| 89712598|            35676000|              82335| 1300182.579710145|
+-------------+------+---------+--------------------+----